### Notebook setup

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

### Create Engine/Session

In [2]:
import sqlalchemy as sa
import sqlalchemy.orm

engine = sa.create_engine(
    "cockroachdb://root@cockroach:26257/defaultdb?sslmode=disable", echo=True
)
engine

Engine(cockroachdb://root@cockroach:26257/defaultdb?sslmode=disable)

<IPython.core.display.Javascript object>

In [3]:
LocalSession = sa.orm.sessionmaker(engine)
LocalSession

sessionmaker(class_='Session', bind=Engine(cockroachdb://root@cockroach:26257/defaultdb?sslmode=disable), autoflush=True, expire_on_commit=True)

<IPython.core.display.Javascript object>

### Cockroachdb cluster id

In [4]:
with LocalSession() as session:
    statement = sa.text(
        "SELECT value FROM crdb_internal.node_build_info WHERE field = 'ClusterID';"
    )
    result = session.execute(statement)

result.first()

2021-11-09 16:10:53,405 INFO sqlalchemy.engine.Engine select current_schema()
2021-11-09 16:10:53,407 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-09 16:10:53,411 INFO sqlalchemy.engine.Engine select version()
2021-11-09 16:10:53,412 INFO sqlalchemy.engine.Engine [generated in 0.00110s] {}
2021-11-09 16:10:53,489 INFO sqlalchemy.engine.Engine SELECT crdb_internal.increment_feature_counter(%(val)s)
2021-11-09 16:10:53,490 INFO sqlalchemy.engine.Engine [generated in 0.00090s] {'val': 'sqlalchemy-cockroachdb 1.4.3.dev0'}
2021-11-09 16:10:53,491 INFO sqlalchemy.engine.Engine SELECT crdb_internal.increment_feature_counter(%(val)s)
2021-11-09 16:10:53,492 INFO sqlalchemy.engine.Engine [cached since 0.002882s ago] {'val': 'sqlalchemy 2.0'}
2021-11-09 16:10:53,494 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:10:53,496 INFO sqlalchemy.engine.Engine SELECT value FROM crdb_internal.node_build_info WHERE field = 'ClusterID';
2021-11-09 16:10:53,497 INFO sqlalchemy.engine.Engi

('74642f88-2b37-4537-b7e2-ad70560857cd',)

<IPython.core.display.Javascript object>

### Drop/Create tables

In [5]:
from models import Base, User, Message

Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

2021-11-09 16:10:53,948 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:10:53,949 INFO sqlalchemy.engine.Engine SELECT table_name FROM information_schema.tables WHERE table_schema=%(schema)s
2021-11-09 16:10:53,950 INFO sqlalchemy.engine.Engine [generated in 0.00086s] {'schema': 'public'}
2021-11-09 16:10:53,956 INFO sqlalchemy.engine.Engine SELECT table_name FROM information_schema.tables WHERE table_schema=%(schema)s
2021-11-09 16:10:53,957 INFO sqlalchemy.engine.Engine [cached since 0.007688s ago] {'schema': 'public'}
2021-11-09 16:10:53,960 INFO sqlalchemy.engine.Engine 
DROP TABLE message
2021-11-09 16:10:53,961 INFO sqlalchemy.engine.Engine [no key 0.00118s] {}
2021-11-09 16:10:54,573 INFO sqlalchemy.engine.Engine 
DROP TABLE "user"
2021-11-09 16:10:54,574 INFO sqlalchemy.engine.Engine [no key 0.00094s] {}
2021-11-09 16:10:54,578 INFO sqlalchemy.engine.Engine COMMIT
2021-11-09 16:11:00,314 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:11:00,315 INFO sq

<IPython.core.display.Javascript object>

### Add data

Nick, Eli, and Paul sing 99 bottles of beer...

In [6]:
users = [User(name="Nick"), User(name="Eli"), User(name="Paul")]
users

[User(id=None, name='Nick'),
 User(id=None, name='Eli'),
 User(id=None, name='Paul')]

<IPython.core.display.Javascript object>

In [7]:
with LocalSession() as session:
    session.add_all(users)
    session.commit()

2021-11-09 16:11:03,720 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:11:03,722 INFO sqlalchemy.engine.Engine INSERT INTO "user" (name) VALUES (%(name)s) RETURNING "user".id
2021-11-09 16:11:03,723 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ({'name': 'Nick'}, {'name': 'Eli'}, {'name': 'Paul'})
2021-11-09 16:11:04,480 INFO sqlalchemy.engine.Engine COMMIT


<IPython.core.display.Javascript object>

In [8]:
import itertools

cycle = itertools.cycle(users)
messages = []

for i in reversed(range(100)):
    content = f"{i} bottles of beer on the wall..."
    user = next(cycle)
    message = Message(user=user, content=content)
    messages.append(message)

with LocalSession() as session:
    session.add_all(messages)
    session.commit()

2021-11-09 16:11:04,943 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:11:04,947 INFO sqlalchemy.engine.Engine SELECT "user".id AS user_id, "user".name AS user_name 
FROM "user" 
WHERE "user".id = %(pk_1)s
2021-11-09 16:11:04,948 INFO sqlalchemy.engine.Engine [generated in 0.00110s] {'pk_1': 709112183126392833}
2021-11-09 16:11:05,304 INFO sqlalchemy.engine.Engine SELECT "user".id AS user_id, "user".name AS user_name 
FROM "user" 
WHERE "user".id = %(pk_1)s
2021-11-09 16:11:05,305 INFO sqlalchemy.engine.Engine [cached since 0.3582s ago] {'pk_1': 709112183126425601}
2021-11-09 16:11:05,308 INFO sqlalchemy.engine.Engine SELECT "user".id AS user_id, "user".name AS user_name 
FROM "user" 
WHERE "user".id = %(pk_1)s
2021-11-09 16:11:05,309 INFO sqlalchemy.engine.Engine [cached since 0.3623s ago] {'pk_1': 709112183126327297}
2021-11-09 16:11:05,316 INFO sqlalchemy.engine.Engine INSERT INTO message (content, user_id) VALUES (%(content)s, %(user_id)s) RETURNING message.id
2021-11

<IPython.core.display.Javascript object>

### Query for data

#### Count query

In [9]:
### Expect to see 3 users here

with LocalSession() as session:
    statement = sa.select(sa.func.count(User.id))
    results = session.scalar(statement)

results

2021-11-09 16:11:07,269 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:11:07,273 INFO sqlalchemy.engine.Engine SELECT count("user".id) AS count_1 
FROM "user"
2021-11-09 16:11:07,275 INFO sqlalchemy.engine.Engine [generated in 0.00237s] {}
2021-11-09 16:11:07,279 INFO sqlalchemy.engine.Engine ROLLBACK


3

<IPython.core.display.Javascript object>

In [10]:
### Expect to see 100 messages here

with LocalSession() as session:
    statement = sa.select(sa.func.count(Message.id))
    results = session.scalar(statement)

results

2021-11-09 16:11:07,603 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:11:07,605 INFO sqlalchemy.engine.Engine SELECT count(message.id) AS count_1 
FROM message
2021-11-09 16:11:07,606 INFO sqlalchemy.engine.Engine [generated in 0.00095s] {}
2021-11-09 16:11:07,609 INFO sqlalchemy.engine.Engine ROLLBACK


100

<IPython.core.display.Javascript object>

#### Query users with messages

In [11]:
with LocalSession() as session:
    statement = sa.select(User).options(sa.orm.joinedload(User.messages))
    results = session.execute(statement)

results

2021-11-09 16:11:07,978 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:11:07,984 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".name, message_1.id AS id_1, message_1.content, message_1.user_id 
FROM "user" LEFT OUTER JOIN message AS message_1 ON "user".id = message_1.user_id
2021-11-09 16:11:07,985 INFO sqlalchemy.engine.Engine [generated in 0.00157s] {}
2021-11-09 16:11:07,989 INFO sqlalchemy.engine.Engine ROLLBACK


<IPython.core.display.Javascript object>

In [12]:
with LocalSession() as session:
    statement = sa.select(User).options(sa.orm.joinedload(User.messages))
    results = session.execute(statement)
    users = results.unique().scalars().all()
users

2021-11-09 16:11:08,176 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:11:08,177 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".name, message_1.id AS id_1, message_1.content, message_1.user_id 
FROM "user" LEFT OUTER JOIN message AS message_1 ON "user".id = message_1.user_id
2021-11-09 16:11:08,178 INFO sqlalchemy.engine.Engine [cached since 0.1945s ago] {}
2021-11-09 16:11:08,184 INFO sqlalchemy.engine.Engine ROLLBACK


[User(name='Nick', id=709112183126327297, messages=[Message(id=709112187769880577, user_id=709112183126327297, content='99 bottles of beer on the wall...'), Message(id=709112187773157377, user_id=709112183126327297, content='0 bottles of beer on the wall...'), Message(id=709112187773059073, user_id=709112183126327297, content='3 bottles of beer on the wall...'), Message(id=709112187772960769, user_id=709112183126327297, content='6 bottles of beer on the wall...'), Message(id=709112187772862465, user_id=709112183126327297, content='9 bottles of beer on the wall...'), Message(id=709112187772764161, user_id=709112183126327297, content='12 bottles of beer on the wall...'), Message(id=709112187772665857, user_id=709112183126327297, content='15 bottles of beer on the wall...'), Message(id=709112187772567553, user_id=709112183126327297, content='18 bottles of beer on the wall...'), Message(id=709112187772469249, user_id=709112183126327297, content='21 bottles of beer on the wall...'), Message

<IPython.core.display.Javascript object>

In [13]:
users[0].name

'Nick'

<IPython.core.display.Javascript object>

In [14]:
users[0].messages[:5]

[Message(id=709112187769880577, user_id=709112183126327297, content='99 bottles of beer on the wall...'),
 Message(id=709112187773157377, user_id=709112183126327297, content='0 bottles of beer on the wall...'),
 Message(id=709112187773059073, user_id=709112183126327297, content='3 bottles of beer on the wall...'),
 Message(id=709112187772960769, user_id=709112183126327297, content='6 bottles of beer on the wall...'),
 Message(id=709112187772862465, user_id=709112183126327297, content='9 bottles of beer on the wall...')]

<IPython.core.display.Javascript object>

In [15]:
for m in users[0].messages[:5]:
    print(m.content)

99 bottles of beer on the wall...
0 bottles of beer on the wall...
3 bottles of beer on the wall...
6 bottles of beer on the wall...
9 bottles of beer on the wall...


<IPython.core.display.Javascript object>

#### Chained statement example

In [16]:
with LocalSession() as session:
    statement = sa.select(User)
    statement = statement.where(User.name == "Eli")
    statement = statement.options(sa.orm.selectinload(User.messages))
    results = session.execute(statement)
    user = results.scalars().first()

user

2021-11-09 16:11:09,287 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:11:09,290 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".name 
FROM "user" 
WHERE "user".name = %(name_1)s
2021-11-09 16:11:09,290 INFO sqlalchemy.engine.Engine [generated in 0.00074s] {'name_1': 'Eli'}
2021-11-09 16:11:09,296 INFO sqlalchemy.engine.Engine SELECT message.user_id AS message_user_id, message.id AS message_id, message.content AS message_content 
FROM message 
WHERE message.user_id IN (%(primary_keys_1)s)
2021-11-09 16:11:09,297 INFO sqlalchemy.engine.Engine [generated in 0.00159s] {'primary_keys_1': 709112183126392833}
2021-11-09 16:11:09,302 INFO sqlalchemy.engine.Engine ROLLBACK


User(name='Eli', id=709112183126392833, messages=[Message(id=709112187769946113, user_id=709112183126392833, content='98 bottles of beer on the wall...'), Message(id=709112187770044417, user_id=709112183126392833, content='95 bottles of beer on the wall...'), Message(id=709112187770142721, user_id=709112183126392833, content='92 bottles of beer on the wall...'), Message(id=709112187770241025, user_id=709112183126392833, content='89 bottles of beer on the wall...'), Message(id=709112187770339329, user_id=709112183126392833, content='86 bottles of beer on the wall...'), Message(id=709112187770437633, user_id=709112183126392833, content='83 bottles of beer on the wall...'), Message(id=709112187770535937, user_id=709112183126392833, content='80 bottles of beer on the wall...'), Message(id=709112187770634241, user_id=709112183126392833, content='77 bottles of beer on the wall...'), Message(id=709112187770732545, user_id=709112183126392833, content='74 bottles of beer on the wall...'), Messa

<IPython.core.display.Javascript object>

In [17]:
user.name

'Eli'

<IPython.core.display.Javascript object>

In [18]:
for m in user.messages[:5]:
    print(m.content)

98 bottles of beer on the wall...
95 bottles of beer on the wall...
92 bottles of beer on the wall...
89 bottles of beer on the wall...
86 bottles of beer on the wall...


<IPython.core.display.Javascript object>

### Using classmethod queries

In [19]:
with LocalSession() as session:
    messages = Message.from_user(session, "Eli")

len(messages)

2021-11-09 16:11:10,212 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:11:10,214 INFO sqlalchemy.engine.Engine SELECT message.id, message.content, message.user_id 
FROM message JOIN "user" ON "user".id = message.user_id 
WHERE "user".name = %(name_1)s
2021-11-09 16:11:10,214 INFO sqlalchemy.engine.Engine [generated in 0.00070s] {'name_1': 'Eli'}
2021-11-09 16:11:10,220 INFO sqlalchemy.engine.Engine ROLLBACK


33

<IPython.core.display.Javascript object>

In [20]:
messages[:5]

[Message(id=709112187769946113, user_id=709112183126392833, content='98 bottles of beer on the wall...'),
 Message(id=709112187770044417, user_id=709112183126392833, content='95 bottles of beer on the wall...'),
 Message(id=709112187770142721, user_id=709112183126392833, content='92 bottles of beer on the wall...'),
 Message(id=709112187770241025, user_id=709112183126392833, content='89 bottles of beer on the wall...'),
 Message(id=709112187770339329, user_id=709112183126392833, content='86 bottles of beer on the wall...')]

<IPython.core.display.Javascript object>